In [5]:
from __future__ import division

import jqdata               #导入聚宽函数库
from six import StringIO    #使用聚宽readfile函数

import talib

import time

import datetime


import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import statsmodels
from statsmodels.tsa.stattools import coint

# just set the seed for the random number generator

import matplotlib.pyplot as plt

In [6]:
def get_hold_list(index_sec ="000300.XSHG", year = 2010,cor_pe = 1,cor_pb = 1,cor_roe = 1,cor_inc = 1,cor_profit = 1,hold_len = 30):
    
    # 取得获取时间
    operate_date = str(year) + "-8-1"
    fundament_date = str(year - 1)
    
   
    # 获得当时的指数列表
    stocks = get_index_stocks(index_sec,date= operate_date)

    # 获取基本面信息
    q = query(
        indicator.code,
        indicator.roe,
        indicator.net_profit_margin,
        valuation.pe_ratio,
        valuation.pb_ratio,
        indicator.inc_net_profit_to_shareholders_year_on_year
        ).filter(
        valuation.code.in_(stocks))

    fundamental_df = get_fundamentals(q, statDate = fundament_date)
    # 设置索引
    fundamental_df =  fundamental_df.set_index('code')
    
    # 删除Nan数据
    fundamental_df = fundamental_df.dropna(axis = 0)
    
    # 删除PE小于0的数据
    fundamental_df.drop(fundamental_df[fundamental_df['pe_ratio']<0].index,inplace=True,axis = 0)
    
    # 删除ROE小于0的数据
    fundamental_df.drop(fundamental_df[fundamental_df['roe']<0].index,inplace=True,axis = 0)
    
    # 删除净利率小于0的数据
    fundamental_df.drop(fundamental_df[fundamental_df['net_profit_margin']<0].index,inplace=True,axis = 0)

    # 排名列表
    fundamental_df =  pd.concat([fundamental_df,fundamental_df.rank(). \
                                 rename(columns={"roe": "roe_rank",  \
                                                 "net_profit_margin":  "net_profit_margin_rank", \
                                                 "pb_ratio": "pb_ratio_rank",  \
                                                 "pe_ratio": "pe_ratio_rank", \
                                                 "inc_net_profit_to_shareholders_year_on_year": "inc_net_profit_to_shareholders_year_on_year_rank" \
                                                })],axis=1)
    

    
    fundamental_df['roe_score'] = cor_roe * fundamental_df['roe_rank']
    fundamental_df['net_profit_margin_score'] = cor_profit * fundamental_df['net_profit_margin_rank']
    fundamental_df['pb_score'] = cor_pb * fundamental_df['pb_ratio_rank']
    fundamental_df['pe_score'] = cor_pe * fundamental_df['pe_ratio_rank']
    fundamental_df['inc_net_profit_to_shareholders_year_on_year_score'] = cor_inc * fundamental_df['inc_net_profit_to_shareholders_year_on_year_rank']
    
    
    fundamental_df['rank_score'] = fundamental_df['roe_score'] + \
                                    fundamental_df['net_profit_margin_score'] + \
                                    fundamental_df['pb_score'] + \
                                    fundamental_df['pe_score'] + \
                                    fundamental_df['inc_net_profit_to_shareholders_year_on_year_score']
                    
    fundamental_df['rank_total'] = fundamental_df['rank_score'].rank()

    return list(fundamental_df[fundamental_df['rank_total'] > (fundamental_df['rank_total'].shape[0] - hold_len)].index)


In [7]:
def cal_market_return(index_sec ="000300.XSHG",year = 2010):
    begin_date =  str(year)+"-8-1"
    end_date = str(year+1)+"-8-1"
    
    # 获取市场收益
    market_price = get_price(index_sec, start_date = begin_date, end_date= end_date, frequency='daily', fields='close',fq = "pre")
    market_yeild_rate = market_price['close'][-1]/market_price['close'][0]
 
    return market_yeild_rate

In [8]:
def cal_portfolio_return(sec_list ,year = 2010):
    begin_date =  str(year)+"-8-1"
    end_date = str(year+1)+"-8-1"
    
    # 计算策略收益
    yield_rate_portfolio = 0
    for sec in sec_list:
        stock_price = get_price(sec, start_date = begin_date, end_date= end_date, frequency='daily', fields='close',fq = "pre")
        yield_rate_portfolio += stock_price['close'][-1]/stock_price['close'][0]/len(sec_list)

    return yield_rate_portfolio

In [9]:

# 确定指数参数
index_sec = "000300.XSHG"

#存储参数的数
para_list = []


market_ret = {}
for year in range(2008,2017):
    # 生成提示信息
    market_ret[year] = cal_market_return(index_sec,year)

    
count = 0
total_count = 25 * 11 * 11 * 11 * 11 * 11
# 开始构建测试循环
for hold_len in range(30,31):
    for cor_pe in range (-5,6):
        for cor_pb in range (-5,6):
            for cor_roe in range (-5,6):
                for cor_inc in range (-5,6):
                    for cor_profit in range (-5,6):
                        
                        print "完成%.4f%%"%(float(count*100/10000000))
                        
                        # 确定存储市场、组合收益率的dict
                        portfolio_ret = {}
                        
                        # 确定年度范围
                        portfolio_ret_total = 1
                        
                        for year in range(2008,2017):
                            # 生成提示信息
                            hold_list = get_hold_list(index_sec =index_sec, year = year,cor_pe = cor_pe,cor_pb = cor_pb,cor_roe = cor_roe,cor_inc = cor_inc,cor_profit = cor_profit,hold_len = hold_len)
                            portfolio_ret[year] = cal_portfolio_return(hold_list ,year = year)
                            portfolio_ret_total *= portfolio_ret[year]
                            
                            
                        verify_flag = True
                        
                        for keys,value in market_ret.iteritems():
                            if portfolio_ret[keys] < market_ret[keys]:
                                verify_flag = False
                        
                        if verify_flag:
                            print "有效参数！cor_pe：%s，cor_pb：%s，cor_roe：%s，cor_inc：%s，cor_profit：%s，hold_len：%s"%(str(cor_pe),str(cor_pb),str(cor_roe),str(cor_inc),str(cor_profit),str(hold_len))
                            print market_ret
                            print portfolio_ret
                            para_list.append([cor_pe,cor_pb,cor_roe,cor_inc,cor_profit,hold_len,portfolio_ret_total])
                            print para_list
                            
                        count += 1

完成0.0000%
完成0.0000%
完成0.0000%
完成0.0000%
完成0.0000%
完成0.0001%
完成0.0001%
完成0.0001%
完成0.0001%
完成0.0001%
完成0.0001%
完成0.0001%
完成0.0001%
完成0.0001%
完成0.0001%
完成0.0001%
完成0.0002%
完成0.0002%
完成0.0002%
完成0.0002%
完成0.0002%
完成0.0002%
完成0.0002%
完成0.0002%
完成0.0002%
完成0.0003%
完成0.0003%
完成0.0003%
完成0.0003%
完成0.0003%
完成0.0003%
完成0.0003%
完成0.0003%
完成0.0003%
完成0.0003%
完成0.0003%
完成0.0004%
完成0.0004%
完成0.0004%
完成0.0004%
完成0.0004%
完成0.0004%
完成0.0004%
完成0.0004%
完成0.0004%
完成0.0004%
完成0.0005%
完成0.0005%
完成0.0005%
完成0.0005%
完成0.0005%
完成0.0005%
完成0.0005%
完成0.0005%
完成0.0005%
完成0.0006%
完成0.0006%
完成0.0006%
完成0.0006%
完成0.0006%
完成0.0006%
完成0.0006%
完成0.0006%
完成0.0006%
完成0.0006%
完成0.0006%
完成0.0007%
完成0.0007%
完成0.0007%
完成0.0007%
完成0.0007%
完成0.0007%
完成0.0007%
完成0.0007%
完成0.0007%
完成0.0008%
完成0.0008%
完成0.0008%
完成0.0008%
完成0.0008%
完成0.0008%
完成0.0008%
完成0.0008%
完成0.0008%
完成0.0008%
完成0.0008%
完成0.0009%
完成0.0009%
完成0.0009%
完成0.0009%
完成0.0009%
完成0.0009%
完成0.0009%
完成0.0009%
完成0.0009%
完成0.0009%
完成0.0010%
完成0.0010%
完成0.0010%
完成0.0010%


完成0.0082%
完成0.0082%
完成0.0082%
完成0.0082%
完成0.0082%
完成0.0082%
完成0.0083%
完成0.0083%
完成0.0083%
完成0.0083%
完成0.0083%
完成0.0083%
完成0.0083%
完成0.0083%
完成0.0083%
完成0.0083%
完成0.0083%
完成0.0084%
完成0.0084%
完成0.0084%
完成0.0084%
完成0.0084%
完成0.0084%
完成0.0084%
完成0.0084%
完成0.0084%
完成0.0084%
完成0.0085%
完成0.0085%
完成0.0085%
完成0.0085%
完成0.0085%
完成0.0085%
完成0.0085%
完成0.0085%
完成0.0085%
完成0.0086%
完成0.0086%
完成0.0086%
完成0.0086%
完成0.0086%
完成0.0086%
完成0.0086%
完成0.0086%
完成0.0086%
完成0.0086%
完成0.0086%
完成0.0087%
完成0.0087%
完成0.0087%
完成0.0087%
完成0.0087%
完成0.0087%
完成0.0087%
完成0.0087%
完成0.0087%
完成0.0088%
完成0.0088%
完成0.0088%
完成0.0088%
完成0.0088%
完成0.0088%
完成0.0088%
完成0.0088%
完成0.0088%
完成0.0088%
完成0.0089%
完成0.0089%
完成0.0089%
完成0.0089%
完成0.0089%
完成0.0089%
完成0.0089%
完成0.0089%
完成0.0089%
完成0.0089%
完成0.0089%
完成0.0090%
完成0.0090%
完成0.0090%
完成0.0090%
完成0.0090%
完成0.0090%
完成0.0090%
完成0.0090%
完成0.0090%
完成0.0091%
完成0.0091%
完成0.0091%
完成0.0091%
完成0.0091%
完成0.0091%
完成0.0091%
完成0.0091%
完成0.0091%
完成0.0091%
完成0.0092%
完成0.0092%
完成0.0092%
完成0.0092%


完成0.0158%
完成0.0158%
完成0.0158%
完成0.0159%
完成0.0159%
完成0.0159%
完成0.0159%
完成0.0159%
完成0.0159%
完成0.0159%
完成0.0159%
完成0.0159%
完成0.0159%
完成0.0160%
完成0.0160%
完成0.0160%
完成0.0160%
完成0.0160%
完成0.0160%
完成0.0160%
完成0.0160%
完成0.0160%
完成0.0160%
完成0.0161%
完成0.0161%
完成0.0161%
完成0.0161%
完成0.0161%
完成0.0161%
完成0.0161%
完成0.0161%
完成0.0161%
完成0.0162%
完成0.0162%
完成0.0162%
完成0.0162%
完成0.0162%
完成0.0162%
完成0.0162%
完成0.0162%
完成0.0162%
完成0.0162%
完成0.0163%
完成0.0163%
完成0.0163%
完成0.0163%
完成0.0163%
完成0.0163%
完成0.0163%
完成0.0163%
完成0.0163%
完成0.0163%
完成0.0163%
完成0.0164%
完成0.0164%
完成0.0164%
完成0.0164%
完成0.0164%
完成0.0164%
完成0.0164%
完成0.0164%
完成0.0164%
完成0.0164%
完成0.0165%
完成0.0165%
完成0.0165%
完成0.0165%
完成0.0165%
完成0.0165%
完成0.0165%
完成0.0165%
完成0.0165%
完成0.0165%
完成0.0166%
完成0.0166%
完成0.0166%
完成0.0166%
完成0.0166%
完成0.0166%
完成0.0166%
完成0.0166%
完成0.0166%
完成0.0167%
完成0.0167%
完成0.0167%
完成0.0167%
完成0.0167%
完成0.0167%
完成0.0167%
完成0.0167%
完成0.0167%
完成0.0167%
完成0.0168%
完成0.0168%
完成0.0168%
完成0.0168%
完成0.0168%
完成0.0168%
完成0.0168%
完成0.0168%


完成0.0234%
完成0.0234%
完成0.0235%
完成0.0235%
完成0.0235%
完成0.0235%
完成0.0235%
完成0.0235%
完成0.0235%
完成0.0235%
完成0.0235%
完成0.0236%
完成0.0236%
完成0.0236%
完成0.0236%
完成0.0236%
完成0.0236%
完成0.0236%
完成0.0236%
完成0.0236%
完成0.0236%
完成0.0237%
完成0.0237%
完成0.0237%
完成0.0237%
完成0.0237%
完成0.0237%
完成0.0237%
完成0.0237%
完成0.0237%
完成0.0237%
完成0.0238%
完成0.0238%
完成0.0238%
完成0.0238%
完成0.0238%
完成0.0238%
完成0.0238%
完成0.0238%
完成0.0238%
完成0.0238%
完成0.0238%
完成0.0239%
完成0.0239%
完成0.0239%
完成0.0239%
完成0.0239%
完成0.0239%
完成0.0239%
完成0.0239%
完成0.0239%
完成0.0239%
完成0.0240%
完成0.0240%
完成0.0240%
完成0.0240%
完成0.0240%
完成0.0240%
完成0.0240%
完成0.0240%
完成0.0240%
完成0.0240%
完成0.0241%
完成0.0241%
完成0.0241%
完成0.0241%
完成0.0241%
完成0.0241%
完成0.0241%
完成0.0241%
完成0.0241%
完成0.0242%
完成0.0242%
完成0.0242%
完成0.0242%
完成0.0242%
完成0.0242%
完成0.0242%
完成0.0242%
完成0.0242%
完成0.0242%
完成0.0243%
完成0.0243%
完成0.0243%
完成0.0243%
完成0.0243%
完成0.0243%
完成0.0243%
完成0.0243%
完成0.0243%
完成0.0243%
完成0.0244%
完成0.0244%
完成0.0244%
完成0.0244%
完成0.0244%
完成0.0244%
完成0.0244%
完成0.0244%
完成0.0244%


完成0.0316%
完成0.0316%
完成0.0316%
完成0.0317%
完成0.0317%
完成0.0317%
完成0.0317%
完成0.0317%
完成0.0317%
完成0.0317%
完成0.0317%
完成0.0317%
完成0.0318%
完成0.0318%
完成0.0318%
完成0.0318%
完成0.0318%
完成0.0318%
完成0.0318%
完成0.0318%
完成0.0318%
完成0.0318%
完成0.0319%
完成0.0319%
完成0.0319%
完成0.0319%
完成0.0319%
完成0.0319%
完成0.0319%
完成0.0319%
完成0.0319%
完成0.0319%
完成0.0319%
完成0.0320%
完成0.0320%
完成0.0320%
完成0.0320%
完成0.0320%
完成0.0320%
完成0.0320%
完成0.0320%
完成0.0320%
完成0.0321%
完成0.0321%
完成0.0321%
完成0.0321%
完成0.0321%
完成0.0321%
完成0.0321%
完成0.0321%
完成0.0321%
完成0.0321%
完成0.0321%
完成0.0322%
完成0.0322%
完成0.0322%
完成0.0322%
完成0.0322%
完成0.0322%
完成0.0322%
完成0.0322%
完成0.0322%
完成0.0323%
完成0.0323%
完成0.0323%
完成0.0323%
完成0.0323%
完成0.0323%
完成0.0323%
完成0.0323%
完成0.0323%
完成0.0323%
完成0.0323%
完成0.0324%
完成0.0324%
完成0.0324%
完成0.0324%
完成0.0324%
完成0.0324%
完成0.0324%
完成0.0324%
完成0.0324%
完成0.0324%
完成0.0325%
完成0.0325%
完成0.0325%
完成0.0325%
完成0.0325%
完成0.0325%
完成0.0325%
完成0.0325%
完成0.0325%
完成0.0326%
完成0.0326%
完成0.0326%
完成0.0326%
完成0.0326%
完成0.0326%
完成0.0326%
完成0.0326%


...

完成1.5699%
完成1.5699%
完成1.5700%
完成1.5700%
完成1.5700%
完成1.5700%
完成1.5700%
完成1.5700%
完成1.5700%
完成1.5700%
完成1.5700%
完成1.5700%
完成1.5701%
完成1.5701%
完成1.5701%
完成1.5701%
完成1.5701%
完成1.5701%
完成1.5701%
完成1.5701%
完成1.5701%
完成1.5701%
完成1.5702%
完成1.5702%
完成1.5702%
完成1.5702%
完成1.5702%
完成1.5702%
完成1.5702%
完成1.5702%
完成1.5702%
完成1.5702%
完成1.5703%
完成1.5703%
完成1.5703%
完成1.5703%
完成1.5703%
完成1.5703%
完成1.5703%
完成1.5703%
完成1.5703%
完成1.5703%
完成1.5704%
完成1.5704%
完成1.5704%
完成1.5704%
完成1.5704%
完成1.5704%
完成1.5704%
完成1.5704%
完成1.5704%
完成1.5704%
完成1.5705%
完成1.5705%
完成1.5705%
完成1.5705%
完成1.5705%
完成1.5705%
完成1.5705%
完成1.5705%
完成1.5705%
完成1.5705%
完成1.5706%
完成1.5706%
完成1.5706%
完成1.5706%
完成1.5706%
完成1.5706%
完成1.5706%
完成1.5706%
完成1.5706%
完成1.5707%
完成1.5707%
完成1.5707%
完成1.5707%
完成1.5707%
完成1.5707%
完成1.5707%
完成1.5707%
完成1.5707%
完成1.5707%
完成1.5708%
完成1.5708%
完成1.5708%
完成1.5708%
完成1.5708%
完成1.5708%
完成1.5708%
完成1.5708%
完成1.5708%
完成1.5708%
完成1.5709%
完成1.5709%
完成1.5709%
完成1.5709%
完成1.5709%
完成1.5709%
完成1.5709%
完成1.5709%
完成1.5709%


完成1.5781%
完成1.5781%
完成1.5781%
完成1.5782%
完成1.5782%
完成1.5782%
完成1.5782%
完成1.5782%
完成1.5782%
完成1.5782%
完成1.5782%
完成1.5782%
完成1.5782%
完成1.5783%
完成1.5783%
完成1.5783%
完成1.5783%
完成1.5783%
完成1.5783%
完成1.5783%
完成1.5783%
完成1.5783%
完成1.5783%
完成1.5784%
完成1.5784%
完成1.5784%
完成1.5784%
完成1.5784%
完成1.5784%
完成1.5784%
完成1.5784%
完成1.5784%
完成1.5784%
完成1.5785%
完成1.5785%
完成1.5785%
完成1.5785%
完成1.5785%
完成1.5785%
完成1.5785%
完成1.5785%
完成1.5785%
完成1.5785%
完成1.5786%
完成1.5786%
完成1.5786%
完成1.5786%
完成1.5786%
完成1.5786%
完成1.5786%
完成1.5786%
完成1.5786%
完成1.5787%
完成1.5787%
完成1.5787%
完成1.5787%
完成1.5787%
完成1.5787%
完成1.5787%
完成1.5787%
完成1.5787%
完成1.5787%
完成1.5788%
完成1.5788%
完成1.5788%
完成1.5788%
完成1.5788%
完成1.5788%
完成1.5788%
完成1.5788%
完成1.5788%
完成1.5788%
完成1.5789%
完成1.5789%
完成1.5789%
完成1.5789%
完成1.5789%
完成1.5789%
完成1.5789%
完成1.5789%
完成1.5789%
完成1.5789%
完成1.5790%
完成1.5790%
完成1.5790%
完成1.5790%
完成1.5790%
完成1.5790%
完成1.5790%
完成1.5790%
完成1.5790%
完成1.5790%
完成1.5791%
完成1.5791%
完成1.5791%
完成1.5791%
完成1.5791%
完成1.5791%
完成1.5791%
完成1.5791%


完成1.5863%
完成1.5863%
完成1.5863%
完成1.5863%
完成1.5864%
完成1.5864%
完成1.5864%
完成1.5864%
完成1.5864%
完成1.5864%
完成1.5864%
完成1.5864%
完成1.5864%
完成1.5864%
完成1.5865%
完成1.5865%
完成1.5865%
完成1.5865%
完成1.5865%
完成1.5865%
完成1.5865%
完成1.5865%
完成1.5865%
完成1.5865%
完成1.5866%
完成1.5866%
完成1.5866%
完成1.5866%
完成1.5866%
完成1.5866%
完成1.5866%
完成1.5866%
完成1.5866%
完成1.5866%
完成1.5867%
完成1.5867%
完成1.5867%
完成1.5867%
完成1.5867%
完成1.5867%
完成1.5867%
完成1.5867%
完成1.5867%
完成1.5868%
完成1.5868%
完成1.5868%
完成1.5868%
完成1.5868%
完成1.5868%
完成1.5868%
完成1.5868%
完成1.5868%
完成1.5868%
完成1.5869%
完成1.5869%
完成1.5869%
完成1.5869%
完成1.5869%
完成1.5869%
完成1.5869%
完成1.5869%
完成1.5869%
完成1.5869%
完成1.5870%
完成1.5870%
完成1.5870%
完成1.5870%
完成1.5870%
完成1.5870%
完成1.5870%
完成1.5870%
完成1.5870%
完成1.5870%
完成1.5871%
完成1.5871%
完成1.5871%
完成1.5871%
完成1.5871%
完成1.5871%
完成1.5871%
完成1.5871%
完成1.5871%
完成1.5871%
完成1.5872%
完成1.5872%
完成1.5872%
完成1.5872%
完成1.5872%
完成1.5872%
完成1.5872%
完成1.5872%
完成1.5872%
完成1.5872%
完成1.5873%
完成1.5873%
完成1.5873%
完成1.5873%
完成1.5873%
完成1.5873%
完成1.5873%


完成1.5945%
完成1.5945%
完成1.5945%
完成1.5945%
完成1.5945%
完成1.5946%
完成1.5946%
完成1.5946%
完成1.5946%
完成1.5946%
完成1.5946%
完成1.5946%
完成1.5946%
完成1.5946%
完成1.5946%
完成1.5947%
完成1.5947%
完成1.5947%
完成1.5947%
完成1.5947%
完成1.5947%
完成1.5947%
完成1.5947%
完成1.5947%
完成1.5947%
完成1.5948%
完成1.5948%
完成1.5948%
完成1.5948%
完成1.5948%
完成1.5948%
完成1.5948%
完成1.5948%
完成1.5948%
完成1.5949%
完成1.5949%
完成1.5949%
完成1.5949%
完成1.5949%
完成1.5949%
完成1.5949%
完成1.5949%
完成1.5949%
完成1.5949%
完成1.5950%
完成1.5950%
完成1.5950%
完成1.5950%
完成1.5950%
完成1.5950%
完成1.5950%
完成1.5950%
完成1.5950%
完成1.5950%
完成1.5951%
完成1.5951%
完成1.5951%
完成1.5951%
完成1.5951%
完成1.5951%
完成1.5951%
完成1.5951%
完成1.5951%
完成1.5951%
完成1.5952%
完成1.5952%
完成1.5952%
完成1.5952%
完成1.5952%
完成1.5952%
完成1.5952%
完成1.5952%
完成1.5952%
完成1.5952%
完成1.5953%
完成1.5953%
完成1.5953%
完成1.5953%
完成1.5953%
完成1.5953%
完成1.5953%
完成1.5953%
完成1.5953%
完成1.5953%
完成1.5954%
完成1.5954%
完成1.5954%
完成1.5954%
完成1.5954%
完成1.5954%
完成1.5954%
完成1.5954%
完成1.5954%
完成1.5954%
完成1.5955%
完成1.5955%
完成1.5955%
完成1.5955%
完成1.5955%
完成1.5955%


完成1.6027%
完成1.6027%
完成1.6027%
完成1.6027%
完成1.6027%
完成1.6027%
完成1.6028%
完成1.6028%
完成1.6028%
完成1.6028%
完成1.6028%
完成1.6028%
完成1.6028%
完成1.6028%
完成1.6028%
完成1.6029%
完成1.6029%
完成1.6029%
完成1.6029%
完成1.6029%
完成1.6029%
完成1.6029%
完成1.6029%
完成1.6029%
完成1.6029%
完成1.6030%
完成1.6030%
完成1.6030%
完成1.6030%
完成1.6030%
完成1.6030%
完成1.6030%
完成1.6030%
完成1.6030%
完成1.6030%
完成1.6031%
完成1.6031%
完成1.6031%
完成1.6031%
完成1.6031%
完成1.6031%
完成1.6031%
完成1.6031%
完成1.6031%
完成1.6031%
完成1.6032%
完成1.6032%
完成1.6032%
完成1.6032%
完成1.6032%
完成1.6032%
完成1.6032%
完成1.6032%
完成1.6032%
完成1.6032%
完成1.6033%
完成1.6033%
完成1.6033%
完成1.6033%
完成1.6033%
完成1.6033%
完成1.6033%
完成1.6033%
完成1.6033%
完成1.6033%
完成1.6034%
完成1.6034%
完成1.6034%
完成1.6034%
完成1.6034%
完成1.6034%
完成1.6034%
完成1.6034%
完成1.6034%
完成1.6034%
完成1.6035%
完成1.6035%
完成1.6035%
完成1.6035%
完成1.6035%
完成1.6035%
完成1.6035%
完成1.6035%
完成1.6035%
完成1.6035%
完成1.6036%
完成1.6036%
完成1.6036%
完成1.6036%
完成1.6036%
完成1.6036%
完成1.6036%
完成1.6036%
完成1.6036%
完成1.6036%
完成1.6037%
完成1.6037%
完成1.6037%
完成1.6037%
完成1.6037%


In [10]:
para_list

[[-5, -5, 3, -4, -2, 30, nan],
 [-5, -4, 2, -4, -3, 30, nan],
 [-5, 0, 3, 1, 0, 30, 2.2293917555946487],
 [-4, -5, 3, -4, -1, 30, nan],
 [-4, 1, 1, 0, 0, 30, 2.4305283675033613],
 [-3, -5, 4, -4, -1, 30, nan],
 [-3, -5, 4, -3, -2, 30, nan],
 [-3, -2, 5, -1, 1, 30, 2.182196617870729],
 [-3, 2, -2, -5, 0, 30, nan],
 [-2, -5, 3, -3, -3, 30, nan],
 [-2, 1, 0, -5, 0, 30, nan],
 [-1, -5, 2, -2, -2, 30, nan],
 [-1, -5, 4, -3, -3, 30, nan],
 [-1, -5, 4, -2, -2, 30, nan],
 [-1, -4, 1, -2, -1, 30, nan],
 [-1, 0, -1, -5, 1, 30, nan],
 [0, -5, 3, -1, -2, 30, nan],
 [0, -5, 4, -3, -1, 30, nan],
 [0, -5, 4, -1, -2, 30, nan],
 [0, -4, 2, -1, -1, 30, nan],
 [0, -4, 3, -2, -1, 30, nan],
 [0, -3, 2, -1, -1, 30, nan],
 [0, -1, 1, -5, 1, 30, nan],
 [1, -5, 2, -1, -1, 30, nan],
 [1, -5, 3, 0, -1, 30, nan],
 [1, -5, 4, -1, -2, 30, nan],
 [1, -5, 4, -1, -1, 30, nan],
 [1, -4, 2, -3, 0, 30, nan],
 [2, -5, 2, -2, 0, 30, nan],
 [2, -5, 5, -1, -2, 30, nan],
 [2, -2, 1, -2, 1, 30, 1.9289061032597066],
 [2, -2, 2,

In [11]:
df = pd.DataFrame(para_list)

write_file('df.csv', df.to_csv(), append=False)